In [6]:
import pandas as pd
import numpy as np
import math
import sys
from Data_Grabber import *


In [8]:
def getLink(name):
    '''
    Grabs the link from this site for the Host name of supernovae in CSV,
    returns the parsed page
    '''
    try:
        link = "http://ned.ipac.caltech.edu/cgi-bin/nph-objsearch?"
        inputs = {'objname': name,
				'extend': 'no',
				'hconst': '73.04',
				'omegam': '0.27',
				'omegav': '0.73',
				'corr_z': '1',
				'out_csys': 'Equatorial',
				'out_equinox': 'J2000.0',
				'obj_sort': "RA or Longitude",
				'of': 'pre_text',
				'zv_breaker': '30000.0',
				'list_limit':'10',
				'img_stamp': 'YES'}
        page = requests.get(link, params = inputs)
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup
    except requests.ConnectionError:
        soup = ''
        return soup



In [14]:
def scrapeValues(soup):
	
    try:
        soup = soup.find("a", attrs={'name':'DerivedValues_0'})
        velocities = soup.next_sibling.next_sibling.next_sibling.find("pre")
        Helio = list(velocities.children)[2]
        VGS = list(velocities.children)[16]
        Helio = Helio.lstrip('\n')
        VGS = VGS.lstrip('\n')
        Hvals = [int(s) for s in Helio.split() if s.isdigit()]
        VGSVals = [int(s) for s in VGS.split() if s.isdigit()]
        vels = [Hvals[0],Hvals[1],VGSVals[0],VGSVals[1]]
        return vels
    except: 
        vels = [np.nan,np.nan,np.nan,np.nan]
        return vels

def get_coords(gals):
    """
    Takes list of galaxies and looks up their coordinates by name.
    If no name found: warn, skip, remove galaxy from list

    Returns:
        gals: list of galaxies minus those that weren't found
        start_coord: list of coordinates corresponding to center of galaxies in 'gals'
    """
    start_coord = []
    #bar = FillingCirclesBar('Loading galaxies', max = len(gals))
    try:
        tempCoord = SkyCoord.from_name(str(gals), frame = 'icrs')
        start_coord= tempCoord
            #bar.next()
    except NameResolveError:
        #if gals != str(''):
            #print('\nSkipping',gals,'because it couldn\'t be found.')
            #start_coord= ''
        #else:
            start_coord= np.nan
    #bar.finish()
    return(start_coord)


In [15]:
soup=getLink('UGC 01398')
vels=scrapeValues(soup)
print(vels)


[5216, 7, 5108, 22]


In [13]:
print(soup)


<html><head>
<title>Your NED Search Results</title>
<script>
 function onSelectSDSSOption(sel) {
    if (sel.value == 12) {
   document.getElementById("sdssinfo").action =
    "http://skyserver.sdss.org/dr" + sel.value + "/en/tools/explore/summary.aspx";
   }
    else  {
      document.getElementById("sdssinfo").action =
    "http://cas.sdss.org/dr" + sel.value + "/en/tools/explore/obj.asp";
    }
    }
</script>
<script>
 function onSelectSDSSOptionPos(sel) {
    if (sel.value == 12) {
   document.getElementById("sdssinfoPos").action =
    "http://skyserver.sdss.org/dr" + sel.value + "/en/tools/explore/summary.aspx";
   }
    else  {
     document.getElementById("sdssinfoPos").action =
    "http://cas.sdss.org/dr" + sel.value + "/en/tools/explore/obj.asp";
    }
    }
</script>
<script>
 function onSelectSDSSOptionInt(sel) {
   if (sel.value == 12) {
   document.getElementById("sdssinfoInt").action =
    "http://skyserver.sdss.org/dr" + sel.value + "/en/tools/chart/navi.aspx";
   }
  